# Python and pip setup

In [ ]:
import subprocess
pip_version = subprocess.run(["pip", "--version"], capture_output=True, text=True)
print("Pip Version:", pip_version.stdout.strip())

Pip Version: pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)


In [ ]:
import platform
print("Python Version:", platform.python_version())

Python Version: 3.11.11


# LIBRARIES LOAD

In [ ]:
%pip install contextily
%pip install plotly
%pip install geopandas
%pip install fastkml fiona shapely lxml
%pip install hdbscan
%pip install --upgrade hdbscan scikit-learn
# !pip install hdbscan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from sklearn.model_selection import ParameterGrid
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples
from itertools import product
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import os
import fiona
from zipfile import ZipFile
import hdbscan
import tempfile
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import os
import requests
import tempfile
import geopandas as gpd
import pandas as pd
import fiona
from zipfile import ZipFile
from io import BytesIO

# FUNCTIONS TO PLOT RESULTS

In [ ]:
def plot_data(df,faults_df, style = "carto-positron"):
    fig = go.Figure(go.Scattermap(
        lon = df['x'], lat = df['y'],
        marker = { 'size': 3, 'color': "red"}, name="Earthquakes"))


    fig.add_trace(go.Scattermap(
            lat=faults_df["Latitude"],
            lon=faults_df["Longitude"],
            mode='markers',
            marker=go.scattermap.Marker(
                size=2, color='#3776ab'
            ),name="Faults"
        ))

    fig.update_layout(
        map = {
            #
            'style': style,
            'center': {'lon': df['x'].median(), 'lat': df['y'].median() },
            'zoom': 5.5},
        width=950,  # Set the width of the output image
        height=800,
        showlegend = True)

    fig.show()


## GET THE PARAMETERS SET FROM RESULTS_DF

In [ ]:
def get_param_key(mode, result_df, line_id):
    """
    Generates a tuple of parameters from a row ID in result_df.

    Args:
        result_df (pd.DataFrame): DataFrame containing clustering results.
        line_id (int): Index of the row to extract parameters from.

    Returns:
        tuple: Formatted parameter key as a tuple of (key, value) pairs.
    """
    if line_id not in result_df.index:
        raise ValueError(f"Row ID {line_id} not found in DataFrame.")

    if mode == 'kmeans':
        row = result_df.loc[line_id, ['init', 'max_iter', 'n_clusters', 'n_init']].to_dict()
        param_key = tuple(row.items())

    if mode == 'dbscan':
        row = result_df.loc[line_id, ['eps',  'metric', 'min_samples']].to_dict()
        param_key = tuple(row.items())

    if mode == 'hdbscan':
        row = result_df.loc[line_id, ['metric', 'min_cluster_size']].to_dict()
        param_key = tuple((k, None if pd.isna(v) else v) for k, v in row.items())

    return param_key

## PLOT CLUSTERS

In [ ]:
def plot_data_clusters(results_df, clusters_dict,  faults_df, id, mode, style="carto-positron"):
    """
    Plots clustered data on a map using Plotly (go.Scattermap) with a proper legend.

    Args:
        clusters_dict (dict): Dictionary containing clustering results.
        param_key (tuple): The specific parameter set as a tuple (same format as dictionary keys).
        style (str): Map style (default: "carto-positron").

    Returns:
        None
    """

    param_key = get_param_key(mode, results_df, id)
    if 'data' not in clusters_dict:
        raise KeyError("The dictionary must contain an entry with key 'data' containing the original dataset.")

    if param_key not in clusters_dict:
        raise KeyError(f"Parameters {param_key} not found in clusters_dict.")

    # Retrieve original data
    data = clusters_dict['data'].copy()

    # Retrieve cluster labels
    data['cluster'] = clusters_dict[param_key]

    # Create a scatter map figure
    fig = go.Figure()

    fig.add_trace(go.Scattermap(
            lat=faults_df["Latitude"],
            lon=faults_df["Longitude"],
            mode='markers',
            marker=go.scattermap.Marker(
                size=2, color='#3776ab'
            ),
            name="Faults"
        ))

    # Get unique clusters
    unique_clusters = sorted(data['cluster'].unique())

    # Define a color map for clusters
    color_map = px.colors.qualitative.Set1  # Change color scheme if needed

    # Plot each cluster separately to show legend
    for i, cluster in enumerate(unique_clusters):
        cluster_data = data[data['cluster'] == cluster]

        fig.add_trace(go.Scattermap(
            lon=cluster_data['x'], lat=cluster_data['y'],
            mode="markers",
            marker=dict(
                size=4,
                color=color_map[i % len(color_map)],  # Assign color from color_map
                opacity=0.7
            ),
            text=[f"Cluster {cluster}"] * len(cluster_data),  # Hover text
            name=f"Cluster {cluster}"  # Important: This ensures a legend entry
        ))

    # Format param_key into a more readable title
    formatted_params = ', '.join(f"{key} = {value}" for key, value in param_key)
    # Convert the row into a list of dictionaries, then extract the first one
    param_stat_list = results_df.loc[results_df['index'] == id, ['silhouette_score', 'dbi', 'chs']].to_dict(orient='records')
    param_stat = param_stat_list[0] if param_stat_list else {}
    formatted_stat = ', '.join(f"{key} = {value:0.3f}" for key, value in param_stat.items())
    # Update layout with map style
    fig.update_layout(
        map={
            'style': style,
            'center': {'lon': data['x'].median(), 'lat': data['y'].median()},
            'zoom': 5.5
        },
        width=950,
        height=800,
        showlegend=True,  # Ensures legend is displayed
        title=f"Cluster Visualization for parameters: {formatted_params} <br> with {formatted_stat}"
    )

    fig.show()


## PLOT 2-DIM STATS

In [ ]:
import plotly.express as px

def plot_2d_stat(results_df, x_variable, y_variable, color_variable, title=''):

    hover_data = [col for col in results_df.columns]

    fig = px.scatter(
        results_df,
        x=x_variable,
        y=y_variable,
        color=color_variable,
        color_continuous_scale='plasma',
        size_max=32,
        hover_data=hover_data,  # Include the index in hover data
        title=title
    )
    fig.show()



# DATA LOAD

In [ ]:
# load faults data and transform into dataframe


def read_local_kmz_faults(kmz_folder):
    """Reads KMZ files from a local folder, extracts KML, and returns a DataFrame of fault coordinates."""

    all_fault_points = []  # Store extracted points

    # Get all KMZ files in the folder
    kmz_files = [f for f in os.listdir(kmz_folder) if f.endswith(".kmz")]

    # Temporary folder for extracted KML files
    temp_folder = os.path.join(kmz_folder, "extracted_kml")
    os.makedirs(temp_folder, exist_ok=True)

    for kmz_file in kmz_files:
        kmz_path = os.path.join(kmz_folder, kmz_file)

        try:
            # Extract the KMZ file
            with ZipFile(kmz_path, 'r') as zip_ref:
                zip_ref.extractall(temp_folder)

            # Locate the extracted KML file (most KMZs contain "doc.kml")
            kml_file = os.path.join(temp_folder, "doc.kml")

            # Get available layers
            layers = fiona.listlayers(kml_file)

            # Loop through each layer
            for layer in layers:
                gdf = gpd.read_file(kml_file, driver="KML", layer=layer)

                if gdf.empty:
                    continue

                gdf = gdf.explode(index_parts=True, ignore_index=True)  # Ensure correct format
                gdf["coords"] = gdf["geometry"].apply(lambda geom: list(geom.coords) if geom else None)

                # Store extracted points
                for _, row in gdf.iterrows():
                    if row["coords"]:
                        for coord in row["coords"]:
                            # Handle both (lon, lat) and (lon, lat, alt) cases
                            lon, lat = coord[:2]  # Ignore altitude if present
                            all_fault_points.append({"Longitude": lon, "Latitude": lat, "File": kmz_file})

        except Exception as e:
            print(f"Error processing file {kmz_file}: {e}")
            continue

    # Convert to DataFrame
    faults_df = pd.DataFrame(all_fault_points)

    return faults_df

In [ ]:
def read_github_kmz_faults(github_folder_url, kmz_filenames):
    """
    Reads KMZ files stored in a GitHub repository, extracts KML data, and returns a DataFrame of fault coordinates.

    Args:
        github_folder_url (str): Base URL of the GitHub repository's folder (without filenames).
        kmz_filenames (list): List of KMZ filenames to fetch from GitHub.

    Returns:
        pd.DataFrame: A DataFrame containing fault coordinates from KMZ files.
    """

    all_fault_points = []  # Store extracted points

    # Temporary folder for extracted KML files
    temp_dir = tempfile.mkdtemp()

    for kmz_file in kmz_filenames:
        kmz_url = f"{github_folder_url}/{kmz_file}"  # Construct the GitHub raw file URL

        try:
            # Download the KMZ file from GitHub
            response = requests.get(kmz_url)
            response.raise_for_status()  # Raise error for failed requests

            # Extract KMZ in memory
            with ZipFile(BytesIO(response.content), 'r') as zip_ref:
                zip_ref.extractall(temp_dir)  # Extract to temp directory

            # Locate the extracted KML file (most KMZs contain "doc.kml")
            kml_file = os.path.join(temp_dir, "doc.kml")

            # Get available layers
            layers = fiona.listlayers(kml_file)

            # Loop through each layer
            for layer in layers:
                gdf = gpd.read_file(kml_file, driver="KML", layer=layer)

                if gdf.empty:
                    continue

                gdf = gdf.explode(index_parts=True, ignore_index=True)  # Ensure correct format
                gdf["coords"] = gdf["geometry"].apply(lambda geom: list(geom.coords) if geom else None)

                # Store extracted points
                for _, row in gdf.iterrows():
                    if row["coords"]:
                        for coord in row["coords"]:
                            # Handle both (lon, lat) and (lon, lat, alt) cases
                            lon, lat = coord[:2]  # Ignore altitude if present
                            all_fault_points.append({"Longitude": lon, "Latitude": lat, "File": kmz_file})

        except Exception as e:
            print(f"Error processing file {kmz_file}: {e}")
            continue

    # Convert to DataFrame
    faults_df = pd.DataFrame(all_fault_points)

    return faults_df


In [ ]:
github_kmz_folder = "https://raw.githubusercontent.com/SkurativskaKateryna/AGPH_Earthquake_Clustering_Analysis/main/DATA"
faults_df = read_github_kmz_faults(github_kmz_folder, kmz_filenames=["AFEAD_J38.kmz","AFEAD_J39.kmz","AFEAD_K38.kmz","AFEAD_K39.kmz"])
print(faults_df.head())

   Longitude   Latitude           File
0  43.878518  39.150315  AFEAD_J38.kmz
1  43.905854  39.143344  AFEAD_J38.kmz
2  43.928843  39.133183  AFEAD_J38.kmz
3  43.960966  39.124855  AFEAD_J38.kmz
4  43.980002  39.116527  AFEAD_J38.kmz


In [ ]:
# Example Usage: Set the local folder containing KMZ files
local_kmz_folder = "DATA"  # Change this to your actual folder
faults_df = read_local_kmz_faults(local_kmz_folder)
print(faults_df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'DATA'

In [ ]:
# Load data from GitHub repository
file_name = "https://raw.githubusercontent.com/SkurativskaKateryna/AGPH_Earthquake_Clustering_Analysis/main/DATA/Earthquake_data_base.dat"
data = pd.read_csv(file_name,delimiter="\t", header=None, names=["y", "x"])[["x", "y"]]
print(data.head())

         x        y
0  46.8399  41.4107
1  48.6542  39.8936
2  46.7913  37.8183
3  46.8455  37.8113
4  46.8501  37.8062


In [ ]:
# Load data locally
file_name = "DATA/Earthquake_data_base.dat"
data = pd.read_csv(file_name,delimiter="\t", header=None, names=["y", "x"])[["x", "y"]]
print(data.head())

FileNotFoundError: [Errno 2] No such file or directory: 'DATA/Earthquake_data_base.dat'

# DATA OVERVIEW

In [ ]:
# Print basic statistics
print("Shape:", data.shape)
print("\nBasic Statistics:")
print(data.describe())

Shape: (6097, 2)

Basic Statistics:
                 x            y
count  6097.000000  6097.000000
mean     47.167056    39.789336
std       1.436300     1.306703
min      43.057500    37.560600
25%      46.046700    38.508500
50%      46.854800    39.524600
75%      48.523600    41.001800
max      50.330000    42.481700


The function to plot points from dataframe and it allows to select the style of the map of plotting:


| **Style Name**                  | **Description** |
|---------------------------------|---------------------------------------------------------------|
| `"basic"`                        | Simple, clean map with standard colors and labels. |
| `"carto-darkmatter"`              | Dark-themed map with high contrast, designed for nighttime use. |
| `"carto-darkmatter-nolabels"`     | Dark map without labels, useful for minimalistic designs. |
| `"carto-positron"`                | Light-themed, minimalistic map with soft colors. |
| `"carto-positron-nolabels"`       | Light-colored map without labels, good for data overlays. |
| `"carto-voyager"`                 | General-purpose map with a balance of detail and simplicity. |
| `"carto-voyager-nolabels"`        | Same as Voyager but without labels. |
| `"dark"`                          | Dark mode map similar to `"carto-darkmatter"`, with more vibrant colors. |
| `"light"`                         | Light-themed map similar to `"carto-positron"`, with more contrast. |
| `"open-street-map"`               | Standard OpenStreetMap tiles, free to use. |
| `"outdoors"`                      | Outdoor-style map, includes terrain details, parks, and hiking trails. |
| `"satellite"`                     | Pure satellite imagery without any additional overlays. |
| `"satellite-streets"`             | Satellite imagery with streets, roads, and place names overlaid. |
| `"streets"`                       | Standard street map with clear roads and points of interest. |
| `"white-bg"`                      | Blank white canvas (no map). Useful for custom layers and no external HTTP requests. |


In [ ]:
plot_data(data, faults_df, style = "carto-positron" )

# CLUSTERIZATION

## KMEAN algorithm

In [ ]:
def KMEAN_gridsearch(df):
    # Define parameter grid
    param_grid = {
        'n_clusters': list(range(2,int(np.sqrt(len(df))))),  # Range of clusters
        'init': ['k-means++', 'random'],  # Initialization methods
        'n_init': [ 30],  # Number of random restarts
        'max_iter': [300] #, 500]  # Max iterations
    }

    # Store results
    results = []
    clusters_dict = {}  # Dictionary to store cluster labels
    clusters_dict['data'] = df # Store original data

    # Total number of iterations
    total_combinations = len(list(ParameterGrid(param_grid)))
    current_iteration = 0

    # Grid Search over parameters
    for params in ParameterGrid(param_grid):
        current_iteration += 1

        # Print progress
        print(f"Iteration {current_iteration}/{total_combinations}: Testing parameters {params}")

        # Apply KMeans clustering
        kmeans = KMeans(
            n_clusters=params['n_clusters'],
            init=params['init'],
            n_init=params['n_init'],
            max_iter=params['max_iter'],
            random_state=42
        )

        cluster_labels = kmeans.fit_predict(df)

        # Compute metrics
        inertia = kmeans.inertia_
        silhouette_avg = silhouette_score(df, cluster_labels)
        dbi = davies_bouldin_score(df, cluster_labels)
        chs = calinski_harabasz_score(df, cluster_labels)

        # Compute silhouette scores for each sample
        silhouette_values = silhouette_samples(df, cluster_labels)
        silhouette_per_cluster = {label: [] for label in cluster_labels if label != -1}  # Exclude noise (-1)
        for label, score in zip(cluster_labels, silhouette_values):
            if label != -1:  # Exclude noise points from per-cluster silhouette scores
                silhouette_per_cluster[label].append(score)

        # Print results for each iteration
        print(f"  - Silhouette Score: {silhouette_avg:.4f}, Inertia: {inertia:.2f}, DBI: {dbi:.4f}, CHS: {chs:.4f}")

        # Store results in DataFrame format
        results.append({
            'n_clusters': params['n_clusters'],
            'init': params['init'],
            'n_init': params['n_init'],
            'max_iter': params['max_iter'],
            'inertia': inertia,
            'silhouette_score': silhouette_avg,
            'dbi': dbi,
            'chs': chs,
            'silhouette_per_cluster': silhouette_per_cluster
        })

        # Store cluster labels in dictionary (using tuple of params as key)
        clusters_dict[tuple(params.items())] = cluster_labels

    # Convert results to a DataFrame and sort by best silhouette score
    results_df = pd.DataFrame(results).sort_values(by='silhouette_score', ascending=False).reset_index(drop=True)

    print("\n✅ Grid Search Completed! Returning results.")
    results_df = results_df.reset_index()

    return results_df, clusters_dict


In [ ]:
results_df_kmean, clusters_dict_kmean = KMEAN_gridsearch(data)

Iteration 1/2: Testing parameters {'init': 'k-means++', 'max_iter': 300, 'n_clusters': 2, 'n_init': 30}
  - Silhouette Score: 0.4182, Inertia: 12910.19, DBI: 1.0442, CHS: 4756.1789
Iteration 2/2: Testing parameters {'init': 'random', 'max_iter': 300, 'n_clusters': 2, 'n_init': 30}
  - Silhouette Score: 0.4182, Inertia: 12910.19, DBI: 1.0442, CHS: 4756.1789

✅ Grid Search Completed! Returning results.


In [ ]:
results_df_kmean

,index,n_clusters,init,n_init,max_iter,inertia,silhouette_score,dbi,chs,silhouette_per_cluster
0,0,2,k-means++,30,300,12910.185596,0.418156,1.044154,4756.178855,"{1: [0.0679661841864393, 0.26922396998531023, ..."
1,1,2,random,30,300,12910.190260,0.418156,1.044154,4756.178855,"{0: [0.0679661841864393, 0.26922396998531023, ..."


In [ ]:
plot_data_clusters(results_df_kmean, clusters_dict_kmean, faults_df, id=0, mode='kmeans', style="carto-positron")

In [ ]:
results_df_kmean

,index,n_clusters,init,n_init,max_iter,inertia,silhouette_score,dbi,chs,silhouette_per_cluster
0,0,5,k-means++,30,300,3059.774402,0.541274,0.593147,9917.529044,"{1: [0.5273192203396102, 0.6510312071360105, 0..."
1,1,5,random,30,300,3059.778056,0.541274,0.593147,9917.529044,"{1: [0.5273192203396102, 0.6510312071360105, 0..."
2,2,4,random,30,300,4156.690435,0.528023,0.619451,9199.504864,"{3: [0.520745684558876, 0.6470638093987943, 0...."
3,3,4,k-means++,30,300,4156.685532,0.528023,0.619451,9199.504864,"{1: [0.520745684558876, 0.6470638093987943, 0...."
4,4,6,k-means++,30,300,2597.994720,0.517175,0.694812,9559.283474,"{3: [0.5736379189902604, 0.4555523959065602, 0..."
...,...,...,...,...,...,...,...,...,...,...
147,147,76,random,30,300,164.944299,0.406161,0.779446,11106.930666,"{62: [0.5743134231025165, 0.3144288251423353, ..."
148,148,77,random,30,300,167.676515,0.402922,0.774640,10778.835377,"{22: [0.3348143774984278, 0.36989279263477454,..."
149,149,73,random,30,300,171.836466,0.402520,0.773468,11107.658577,"{6: [-0.04473684941866245, 0.23298907290357762..."
150,150,68,random,30,300,190.674978,0.402504,0.785930,10757.827914,"{10: [0.6277770853112917, 0.30622057769740724,..."


In [ ]:
plot_2d_stat(results_df=results_df_kmean.drop(columns=['silhouette_per_cluster']), x_variable='n_clusters', y_variable='silhouette_score', color_variable='chs', title='KMEAN parameter analysis')
plot_2d_stat(results_df=results_df_kmean.drop(columns=['silhouette_per_cluster']), x_variable='n_clusters', y_variable='chs', color_variable='silhouette_score', title='KMEAN parameter analysis')

## DBSCAN algorithm

In [ ]:
def DBSCAN_gridsearch(df):
    # Define parameter grid
    param_grid = {
        'eps': np.round(np.linspace(0.2, 1.0, num=int((1.0 - 0.2) / 0.1) + 1), 2).tolist(),  # from 0.2 to 1.0 with the step 0.1
        'min_samples': np.linspace(25, 350, num=int((350 - 25) / 25) + 1, dtype=int).tolist(),  # from 25 to 350 with the step 25
        'metric': ['euclidean', 'manhattan']  # Distance metrics
    }

    # Store results
    results = []
    clusters_dict = {}
    clusters_dict['data'] = df  # Store original data

    # Total number of iterations
    total_combinations = len(list(ParameterGrid(param_grid)))
    current_iteration = 0

    # Grid Search over parameters
    for params in ParameterGrid(param_grid):
        current_iteration += 1
        print(f"Iteration {current_iteration}/{total_combinations}: Testing parameters {params}")

        # Apply DBSCAN clustering
        dbscan = DBSCAN(
            eps=params['eps'],
            min_samples=params['min_samples'],
            metric=params['metric']
        )
        cluster_labels = dbscan.fit_predict(df)

        # Ignore models where all points are labeled as noise (-1)
        unique_labels = set(cluster_labels)
        if len(unique_labels) <= 1:
            print("  - Skipping due to all points being noise.")
            continue

        # Compute clustering metrics
        silhouette_avg = silhouette_score(df, cluster_labels) if len(unique_labels) > 1 else -1
        dbi = davies_bouldin_score(df, cluster_labels)
        chs = calinski_harabasz_score(df, cluster_labels)

        # Compute silhouette scores for each sample
        silhouette_values = silhouette_samples(df, cluster_labels)
        silhouette_per_cluster = {label: [] for label in unique_labels if label != -1}  # Exclude noise (-1)
        for label, score in zip(cluster_labels, silhouette_values):
            if label != -1:  # Exclude noise points from per-cluster silhouette scores
                silhouette_per_cluster[label].append(score)

        print(f"  - Number of clusters: {len(unique_labels) - 1}, Silhouette Score: {silhouette_avg:.4f}, DBI: {dbi:.4f}, CHS: {chs:.4f}")

        # Store results
        results.append({
            'eps': params['eps'],
            'min_samples': params['min_samples'],
            'metric': params['metric'],
            'n_clusters': len(unique_labels) - 1,
            'silhouette_score': silhouette_avg,
            'dbi': dbi,
            'chs': chs,
            'silhouette_per_cluster': silhouette_per_cluster
        })

        # Store cluster labels in dictionary
        clusters_dict[tuple(params.items())] = cluster_labels

    # Convert results to a DataFrame and sort by best silhouette score
    results_df = pd.DataFrame(results).sort_values(by='silhouette_score', ascending=False).reset_index(drop=True)
    results_df = results_df.reset_index()

    print("\n✅ Grid Search Completed! Returning results.")

    return results_df, clusters_dict


In [ ]:
results_df_dbscan, clusters_dict_dbscan = DBSCAN_gridsearch(data)

Iteration 1/252: Testing parameters {'eps': 0.2, 'metric': 'euclidean', 'min_samples': 25}
  - Number of clusters: 5, Silhouette Score: 0.0655, DBI: 2.1579, CHS: 881.0124
Iteration 2/252: Testing parameters {'eps': 0.2, 'metric': 'euclidean', 'min_samples': 50}
  - Number of clusters: 14, Silhouette Score: 0.2256, DBI: 1.4597, CHS: 940.7011
Iteration 3/252: Testing parameters {'eps': 0.2, 'metric': 'euclidean', 'min_samples': 75}
  - Number of clusters: 8, Silhouette Score: 0.1925, DBI: 1.1937, CHS: 1120.1219
Iteration 4/252: Testing parameters {'eps': 0.2, 'metric': 'euclidean', 'min_samples': 100}
  - Number of clusters: 7, Silhouette Score: 0.1733, DBI: 1.1856, CHS: 1103.7965
Iteration 5/252: Testing parameters {'eps': 0.2, 'metric': 'euclidean', 'min_samples': 125}
  - Number of clusters: 6, Silhouette Score: 0.1168, DBI: 1.2377, CHS: 1047.4383
Iteration 6/252: Testing parameters {'eps': 0.2, 'metric': 'euclidean', 'min_samples': 150}
  - Number of clusters: 5, Silhouette Score: 0.

In [ ]:
results_df_dbscan

,index,eps,min_samples,metric,n_clusters,silhouette_score,dbi,chs,silhouette_per_cluster
0,0,0.5,250,euclidean,4,0.434387,1.629368,3777.900732,"{0: [0.6209164583230211, 0.7122689794058458, 0..."
1,1,0.6,325,euclidean,3,0.429664,1.892150,3603.519252,"{0: [0.6544387247421847, 0.6819572387043271, 0..."
2,2,0.7,275,manhattan,3,0.426869,1.929672,3471.695926,"{0: [0.6588064519566306, 0.6891507479572998, 0..."
3,3,0.5,275,euclidean,4,0.425849,1.554063,3682.032924,"{0: [0.620886952006477, 0.7253187246745203, 0...."
4,4,0.5,300,euclidean,5,0.425167,1.361163,3534.437622,"{0: [0.6923688225053379, 0.23320296821049824, ..."
...,...,...,...,...,...,...,...,...,...
246,246,0.2,200,euclidean,4,0.028984,1.145029,981.401561,"{0: [0.8768611569557928, 0.8902155888829663, 0..."
247,247,0.3,25,manhattan,2,0.007933,37.464633,27.707808,"{0: [-0.23167953352013432, 0.1442309076093764,..."
248,248,0.2,100,manhattan,8,0.004203,1.225002,671.882957,"{0: [0.6176091114479945, 0.46701648297584114, ..."
249,249,0.2,150,manhattan,5,0.001356,1.146173,806.407595,"{0: [0.8328520938726495, 0.858913520762565, 0...."


In [ ]:
plot_2d_stat(results_df=results_df_dbscan.drop(columns=['silhouette_per_cluster']), x_variable='n_clusters', y_variable='silhouette_score', color_variable='eps', title='DBSCAN parameter analysis')
plot_2d_stat(results_df=results_df_dbscan.drop(columns=['silhouette_per_cluster']), x_variable='n_clusters', y_variable='chs', color_variable='silhouette_score', title='DBSCAN parameter analysis')
plot_2d_stat(results_df=results_df_dbscan.drop(columns=['silhouette_per_cluster']), x_variable='n_clusters', y_variable='dbi', color_variable='silhouette_score', title='DBSCAN parameter analysis')

In [ ]:
plot_data_clusters(results_df_dbscan, clusters_dict_dbscan, faults_df, id=0, mode='dbscan', style="carto-positron")

## HDBSCAN algorithm

In [ ]:
def HDBSCAN_gridsearch(df):
    # Define parameter grid
    param_grid = {
        'min_cluster_size': np.linspace(75, 200, num=int((200 - 75) / 25)+1 , dtype=int).tolist(),
        'metric': ['euclidean', 'manhattan']
    }

    # Store results
    results = []
    clusters_dict = {}
    clusters_dict['data'] = df  # Store original data

    # Total number of iterations
    total_combinations = len(list(ParameterGrid(param_grid)))
    current_iteration = 0

    # Grid Search over parameters
    for params in ParameterGrid(param_grid):
        current_iteration += 1
        print(f"Iteration {current_iteration}/{total_combinations}: Testing parameters {params}")

        # Apply HDBSCAN clustering
        hdb = hdbscan.HDBSCAN(
            min_cluster_size=params['min_cluster_size'],
            metric=params['metric']
        )
        cluster_labels = hdb.fit_predict(df)

        # Ignore models where all points are labeled as noise (-1)
        unique_labels = set(cluster_labels)
        if len(unique_labels) <= 1:
            print("  - Skipping due to all points being noise.")
            continue

        # Compute clustering metrics
        silhouette_avg = silhouette_score(df, cluster_labels) if len(unique_labels) > 1 else -1
        dbi = davies_bouldin_score(df, cluster_labels)
        chs = calinski_harabasz_score(df, cluster_labels)

        # Compute silhouette scores for each sample
        silhouette_values = silhouette_samples(df, cluster_labels)
        silhouette_per_cluster = {label: [] for label in unique_labels if label != -1}  # Exclude noise (-1)
        for label, score in zip(cluster_labels, silhouette_values):
            if label != -1:  # Exclude noise points from per-cluster silhouette scores
                silhouette_per_cluster[label].append(score)

        print(f"  - Number of clusters {len(unique_labels) - 1}, Silhouette Score: {silhouette_avg:.4f}, DBI: {dbi:.4f}, CHS: {chs:.4f}")

        # Store results
        results.append({
            'min_cluster_size': params['min_cluster_size'],
            'metric': params['metric'],
            'n_clusters': len(unique_labels) - 1,
            'silhouette_score': silhouette_avg,
            'dbi': dbi,
            'chs': chs,
            'silhouette_per_cluster': silhouette_per_cluster
        })

        # Store cluster labels in dictionary
        clusters_dict[tuple(params.items())] = cluster_labels

    # Convert results to a DataFrame and sort by best silhouette score
    results_df = pd.DataFrame(results).sort_values(by='silhouette_score', ascending=False).reset_index(drop=True)
    results_df = results_df.reset_index()
    print("\n✅ Grid Search Completed! Returning results.")

    return results_df, clusters_dict


In [ ]:
results_df_hdbscan, clusters_dict_hdbscan = HDBSCAN_gridsearch(data)

Iteration 1/12: Testing parameters {'metric': 'euclidean', 'min_cluster_size': 75}
  - Number of clusters 10, Silhouette Score: 0.1198, DBI: 1.3342, CHS: 810.6305
Iteration 2/12: Testing parameters {'metric': 'euclidean', 'min_cluster_size': 100}
  - Number of clusters 7, Silhouette Score: 0.2450, DBI: 1.2952, CHS: 1297.2641
Iteration 3/12: Testing parameters {'metric': 'euclidean', 'min_cluster_size': 125}
  - Number of clusters 6, Silhouette Score: 0.3096, DBI: 1.3574, CHS: 1723.9699
Iteration 4/12: Testing parameters {'metric': 'euclidean', 'min_cluster_size': 150}
  - Number of clusters 6, Silhouette Score: 0.2867, DBI: 1.3185, CHS: 1612.9201
Iteration 5/12: Testing parameters {'metric': 'euclidean', 'min_cluster_size': 175}
  - Number of clusters 5, Silhouette Score: 0.2656, DBI: 1.3678, CHS: 1754.1745
Iteration 6/12: Testing parameters {'metric': 'euclidean', 'min_cluster_size': 200}
  - Number of clusters 5, Silhouette Score: 0.2413, DBI: 1.3330, CHS: 1621.4616
Iteration 7/12: T

In [ ]:
results_df_hdbscan

,index,min_cluster_size,metric,n_clusters,silhouette_score,dbi,chs,silhouette_per_cluster
0,0,125,manhattan,6,0.333045,1.400336,1861.591754,"{0: [0.5967360091630095, 0.735810893187667, 0...."
1,1,125,euclidean,6,0.309595,1.357435,1723.969907,"{0: [0.722213656535746, 0.290091502278192, 0.5..."
2,2,175,manhattan,5,0.303421,1.360088,1959.393410,"{0: [0.8510499526495351, 0.5103472742388231, 0..."
3,3,150,manhattan,6,0.290786,1.334187,1629.875516,"{0: [0.7394792863916483, 0.2504239615407458, 0..."
4,4,150,euclidean,6,0.286712,1.318479,1612.920066,"{0: [0.7328310492429944, 0.2548128167386004, 0..."
5,5,175,euclidean,5,0.265569,1.367835,1754.174528,"{0: [0.8503770156483813, 0.5101167859573401, 0..."
6,6,100,euclidean,7,0.245006,1.295246,1297.264145,"{0: [0.8029198678013847, 0.4740660743399552, 0..."
7,7,200,euclidean,5,0.241262,1.333011,1621.461567,"{0: [0.8516135380078246, 0.5115414564238911, 0..."
8,8,200,manhattan,5,0.234806,1.376167,1631.672268,"{0: [0.6683958163221203, 0.7987904684421788, 0..."
9,9,100,manhattan,7,0.226486,1.261468,1259.865888,"{0: [0.8042081137010999, 0.47282266395712874, ..."


In [ ]:
plot_2d_stat(results_df=results_df_hdbscan.drop(columns=['silhouette_per_cluster']), x_variable='n_clusters', y_variable='silhouette_score', color_variable='min_cluster_size', title='HDBSCAN parameter analysis')
plot_2d_stat(results_df=results_df_hdbscan.drop(columns=['silhouette_per_cluster']), x_variable='n_clusters', y_variable='chs', color_variable='min_cluster_size', title='HDBSCAN parameter analysis')
plot_2d_stat(results_df=results_df_hdbscan.drop(columns=['silhouette_per_cluster']), x_variable='n_clusters', y_variable='dbi', color_variable='min_cluster_size', title='HDBSCAN parameter analysis')

In [ ]:
plot_data_clusters(results_df_hdbscan, clusters_dict_hdbscan, faults_df, id=6, mode='hdbscan', style="carto-positron")